<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-Notebook" data-toc-modified-id="Prepare-Notebook-1">Prepare Notebook</a></span></li><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-2">Load the Data</a></span></li><li><span><a href="#Detrend-Data" data-toc-modified-id="Detrend-Data-3">Detrend Data</a></span></li><li><span><a href="#Plot-Data" data-toc-modified-id="Plot-Data-4">Plot Data</a></span></li></ul></div>

## Prepare Notebook
[Back to Top](#top)

In [ ]:
from __future__ import print_function
import cdms2
import genutil
import vcs
import requests
import os

filename = 'tas_Amon_IPSL-CM5A-LR_1pctCO2_r1i1p1_185001-198912.nc'
if not os.path.exists(filename):
    r = requests.get("https://cdat.llnl.gov/cdat/sample_data/notebooks/{}".format(filename), stream=True)
    with open(filename,"wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter local_filename keep-alive new chunks
                f.write(chunk)


## Load the Data
[Back to Top](#top)

In [ ]:
tas_file = cdms2.open(filename)

tas = tas_file("tas")
print("Shape: tas {}".format(tas.shape))

## Detrend Data
[Back to Top](#top)

First we will compute the trend, other `time` note that the index of time can be anything, `genutil` will determine its index.

After computation we lose the time axis, also note the uniots, based on units on time axis

In [ ]:
coeff, intercept = genutil.statistics.linearregression(tas, axis="t")

print("Shapes: coeff {}, intercept {}".format(coeff.shape, intercept.shape))
print("Units: {}".format(coeff.units))

Now we need to remove the trend (detrend)

In [ ]:
detrended = tas - tas*coeff

## Plot Data
[Back to Top](#top)

For clarity we will plot the time series only

In [ ]:
ts = genutil.averager(tas, axis='xy')
ts_detrended = genutil.averager(detrended, axis="xy")

In [ ]:
x = vcs.init(bg=True,geometry=(600,400))
red = vcs.create1d()
# Set color
red.linecolor = "red"
# Set y domain
red.datawc_y1 = 55
red.datawc_y2 = 65
# create a copy for blue line
blue = vcs.create1d(source=red)
# reset the color
blue.linecolor = "blue"
x.plot(ts, red)
x.plot(ts_detrended, blue)
#x.clear()
#x.plot(coeff)

In [ ]:
f = cdms2.open("https://vesg.ipsl.upmc.fr/thredds/dodsC/cmip5/output1/IPSL/IPSL-CM5A-LR/1pctCO2/mon/atmos/Amon/r1i1p1/v20110427/tas/tas_Amon_IPSL-CM5A-LR_1pctCO2_r1i1p1_185001-198912.nc")